In [49]:
import sqlite3
import pandas as pd

In [50]:
# Conexión a la base de datos
conn = sqlite3.connect("database.db")
cursor = conn.cursor()

In [51]:
import configparser

# Crear archivo de configuración
config = configparser.ConfigParser()
config["output"] = {
    "ruta_archivo": "resultado_agrupado.txt"  # Puedes cambiar la ruta si deseas
}

# Guardar el archivo config.ini
with open("config.ini", "w") as configfile:
    config.write(configfile)

print("✅ Archivo 'config.ini' creado correctamente.")

✅ Archivo 'config.ini' creado correctamente.


In [52]:
# Paso 1: Crear tabla clientes_sin_duplicados con registros únicos (fecha de vinculación más antigua)
query_clientes_sin_duplicados = """
DROP TABLE IF EXISTS clientes_sin_duplicados;

CREATE TABLE clientes_sin_duplicados AS
SELECT *
FROM clientes c1
WHERE f_vinc = (
    SELECT MIN(f_vinc)
    FROM clientes c2
    WHERE c2.tipo_doc = c1.tipo_doc
      AND c2.num_doc = c1.num_doc
      AND c2.cod_apli_prod = c1.cod_apli_prod
      AND c2.num_cta = c1.num_cta
);
"""
cursor.executescript(query_clientes_sin_duplicados)
conn.commit()
print("✅ Tabla 'clientes_sin_duplicados' creada.")


✅ Tabla 'clientes_sin_duplicados' creada.


In [53]:
# Paso 2: Crear tabla transacciones_acumuladas (suma y suma acumulada por fecha y código transacción)
query_transacciones_acumuladas = """
DROP TABLE IF EXISTS transacciones_acumuladas;

CREATE TABLE transacciones_acumuladas AS
SELECT
    f_aplicacion_trn,
    cod_trn,
    SUM(mnt_trn) AS total_monto,
    SUM(SUM(mnt_trn)) OVER (
        PARTITION BY f_aplicacion_trn
        ORDER BY cod_trn
    ) AS acumulado_monto
FROM transacciones
GROUP BY f_aplicacion_trn, cod_trn;
"""
cursor.executescript(query_transacciones_acumuladas)
conn.commit()
print("✅ Tabla 'transacciones_acumuladas' creada.")


✅ Tabla 'transacciones_acumuladas' creada.


In [56]:
# Paso 3: Crear tabla resultado_agrupado con promedio de monto por num_doc, filtrado y ordenado
query_resultado_agrupado = """
DROP TABLE IF EXISTS resultado_agrupado;

CREATE TABLE resultado_agrupado AS
SELECT
    c.num_doc,
    AVG(t.mnt_trn) AS promedio_monto,
    c.f_vinc
FROM clientes_sin_duplicados c
JOIN transacciones t
  ON c.num_cta = t.num_cta
 AND c.cod_apli_prod = t.cod_apli_prod
GROUP BY c.num_doc, c.f_vinc
HAVING AVG(t.mnt_trn) > 150000
ORDER BY c.f_vinc ASC;
"""

cursor.executescript(query_resultado_agrupado)
conn.commit()

In [57]:
# Leer ruta desde config.ini
config = configparser.ConfigParser()
config.read("config.ini")
ruta_salida = config["output"]["ruta_archivo"]

# Exportar resultado_agrupado
df = pd.read_sql_query("SELECT * FROM resultado_agrupado", conn)
df.to_csv(ruta_salida, sep="\t", index=False)

print(f"✅ Archivo '{ruta_salida}' exportado correctamente.")

✅ Archivo 'resultado_agrupado.txt' exportado correctamente.


In [58]:
# Cerrar conexión
conn.close()

In [ ]:
# Resultado 

# Eliminanos duplicados de clientes manteniendo la fecha de vinculación más antigua.
# Calculamos suma y suma acumulada por fecha y tipo de transacción (tabla transacciones_acumuladas).
# Calculasmos el promedio de monto por cliente, filtraste los que tienen promedio > 150,000 y ordenaste por fecha de vinculación.
# Exportamos el resultado a un archivo txt separado por tabulación.

In [ ]:
# Exportar datos sqlite desde python

conn = sqlite3.connect("database.db")

# Lista de tablas que quieres exportar
tablas = ["clientes_sin_duplicados", "transacciones", "transacciones_acumuladas", "resultado_agrupado"]

# Exportar cada tabla a CSV
for tabla in tablas:
    df = pd.read_sql_query(f"SELECT * FROM {tabla}", conn)
    df.to_csv(f"{tabla}.csv", index=False)
    print(f"Exportado {tabla}.csv")

c3onn.close()